# Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from imblearn import over_sampling

# Importing Data

In [ ]:
!gdown --id 1nn7kCgyManm3pz4p4pOnZRDyYBLx2ujo

In [ ]:
df = pd.read_csv("Travel.csv")

# Stage 0

## Data Understanding

### Problem:
Perusahaan 'Trips & Travel.Com' menawarkan lima jenis produk kepada pelanggan mereka, yaitu Paket Basic, Standard, Deluxe, Super Deluxe, dan King. Namun, berdasarkan data tahun lalu, terlihat tingkat konversi yang rendah, dimana hanya 18% dari pelanggan yang membeli paket yang ditawarkan. Perusahaan belum memanfaatkan informasi yang tersedia dari pelanggan dalam menawarkan produk dan menghubungi pelanggan secara  acak, sehingga biaya marketing menjadi tidak efisien.

### Goal:
Dalam usaha ekspansi user base, perusahaan "Trips & Travel.Com" berencana menawarkan produk baru yaitu Wellness Tourism Package. Dalam meluncurkan produk baru, perusahaan ingin:
* Meningkatkan conversion rate dari penawaran produk baru tersebut.
* Membuat biaya marketing menjadi lebih efisien.

### Objectives:
* Membuat model yang dapat memprediksi pelanggan yang berpotensi membeli paket baru.
* Mengidentifikasi segmentasi pelanggan yang berpotensi membeli produk baru berdasarkan atribut mereka.

### Business Metrics:
* Conversion Rate: Persentase dari pelanggan yang ditawarkan yang akhirnya membeli produk tersebut.
* ROI (Return on Investment): Rasio dari nilai keuntungan terhadap biaya marketing.

# Stage 1

## Defining numerical and categorical column

In [ ]:
num_col = [
    "Age",
    "MonthlyIncome",
    "CityTier",
    "NumberOfTrips",
    "NumberOfPersonVisiting",
    "NumberOfChildrenVisiting",
    "DurationOfPitch",
    "NumberOfFollowups",
    "PreferredPropertyStar",
    "PitchSatisfactionScore",
]

cat_col = [
    "Passport",
    "OwnCar",
    "Gender",
    "MaritalStatus",
    "Occupation",
    "Designation",
    "ProductPitched",
]

target = "ProdTaken"

## Descriptive Statistics

In [ ]:
df[num_col].info()

In [ ]:
df[num_col].describe()

In [ ]:
df[cat_col].info()

In [ ]:
df[cat_col].astype(str).describe()

In [ ]:
for col in cat_col:
    uniq = df[col].astype(str).unique().tolist()
    print(f"Kolom {col} memiliki {len(uniq)} unique Value(s): {', '.join(uniq)}")

In [ ]:
df[[target]].astype(str).describe()

In [ ]:
print("Kolom".ljust(26), end="")
print("Jumlah NaN".ljust(23), end="")
print(">20% of total")

for col, nMt, ratio in zip(
    df.columns, df.isna().sum(), df.isna().sum() / len(df) * 100
):
    if nMt != 0:
        print(f"{col}".ljust(30), end="")
        print(f"{nMt}".ljust(25), end="")
        print("Yes") if ratio > 20 else print("No")

Insight:
1. Age (Usia):
    * Terdapat 226 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 37.6 dan median di 36
    * Usia termuda ada di umur 18
    * Usia tertua ada di 61
2. Monthly Income (Pendapatan bulanan):
    * Terdapat 233 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 23619.9$ dan median di 22347$
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai min yang jauh dari q1 yaitu 1000$ dan nilai max yang jauh dari q3 yaitu 98678$
3. Number Of Trips (Jumlah rerata trip yang dilakukan customer pertahun):
    * Terdapat 140 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 3.2 dan median di 3
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai max yang jauh dari q3 yaitu 22
4. Number of Person Visiting (Jumlah orang dewasa yang ikut trip):
    * Paling umum customer mengajak 2-3 orang dewasa
    * Terdapat juga yang hanya mengajak 1 orang, yang mungkin adalah orang terdekatnya
    * Paling banyak customer pergi bersama 5 orang dewasa lainnya
5. Number of Children Visiting (Jumlah balita yang ikut trip):
    * Terdapat 66 missing values, tidak melampaui 20% dari total data
    * Paling umum customer membawa 1-2 balita
    * Terdapat customer yang sampai membawa 3 balita, namun juga ada yang tidak sama sekali
6. Duration of Pitch (Durasi promosi):
    * Terdapat 251 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 15.5 dan median di 13
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai max yang jauh dari q3 yaitu 127
7. Number of Follow ups (Jumlah tindak lanjut yang dilakukan setelah dilakukannya promosi):
    * Terdapat 45 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 3.7 dan median di 4
    * Terdapat paling banyak 6 kali upaya tindak lanjut dan paling sedikit 1
8. Passport (Apakah customer memiliki passport atau tidak):
    * Sebanyak 71% customer tidak memiliki passport
9. Own Car (Apakah customer memiliki mobil atau tidak):
    * Sebanyak 62% customer memiliki mobil
10. Gender (Jenis kelamin customer):
    * Terdapat 3 gender, bisa jadi ini adalah kesalah tulis (typo)
    * Sebanyak 60% customer berjenis kelamin pria
11. Marital Status (Status pernikahan):
    * Terdapat 4 status berbeda
    * Sebanyak 48% customer berstatus menikah
12. City Tier (Tingkat kelayakan kota yang dihuni customer):
    * Terdapat 3 tingkatan kota
    * Sebanyak 65% customer tinggal di kota tingkat 1
13. Occupation (Tipe pekerjaan yang customer jalani):
    * Terdapat 4 tipe pekerjaan
    * Sebanyak 48% customer bekerja sebagai karyawan
14. Designation (Jabatan pekerjaan dari customer):
    * Terdapat 5 jabatan berbeda
    * Sebanyak 38% customer memiliki jabatan eksekutif
15. Preferred Property Star (Preferensi kelas hotel):
    * Terdapat 26 missing values, tidak melampaui 20% dari total data
    * Normalnya ada di range 1-5 tapi pada dataset hanya berkisar dari 3-5.
    * Sebanyak 61% customer memilih untuk menyewa hotel bintang 3
16. Pitch Satisfaction Score (Tingkat/nilai kepuasan customer terhadap promosi yang diberikan):
    * Terdapat 5 tingkatan berbeda yaitu dari 1-5
    * Sebanyak 30% customer memilih memberi nilai 3
17. Product Pitched (Produk yang dikenalkan pada sesi promosi):
    * Terdapat 5 produk berbeda yang memiliki tingkat yang berbeda
    * Sebanyak 38% customer dikenalkan kepada produk Basic
18. Product Taken (Apakah customer mengambil produk yang dikenalkan atau tidak):
    * Sebanyak 81% tidak mengambil produk yang ditawarkan